### 4-1. NPLM

- 예측 기반 모델
- 기존 언어 모델의 한계를 일부 극복
>학습 데이터에 존재하지 않는 n-gram이 포함된 문장이 나타낼 확률 값을 0으로 부여<br>
단어/문장 간 유사도를 계산할 수 없음

- 단어 시퀀스가 주어졌을 때 다음 단어가 무엇인지 맞추는 과정에서 학습됨
>입력 : n개의 단어 인덱스 값을 확인하고 각 단어에 해당하는 열 벡터를 C(어휘집합크기x차원 수 행렬)에서 참조한 뒤, 하나로 묶어줌<br>
학습 : 정답 단어의 인덱스와 비교해 역전파하는 방식으로 학습<br>
학습이 종료되면 행렬 C를 각 단어에 해당하는 m차원 임베딩으로 사용

- NPLM은 학습해야 하는 파라미터 종류가 많고 크기가 큰 편 -> 학습속도가 느리다!
- but, 문장이 말뭉치에 없어도 문맥이 비슷한 다른 문장을 참고해 확률을 부여한다.

### 4-2. Word2Vec

- 구글 연구팀이 발표한 가장 널리 쓰이고 있는 단어 임베딩 모델
- CBOW와 Skip-gram 두 가지 모델이 제안되었으며, 네거티브 샘플링, 서브 샘플링 등 학습 최적화 기법이 제안됨
> CBOW : 주변에 있는 문맥 단어들을 가지고 타깃 단어 하나를 맞추는 과정에서 학습<br>
Skip-gram : 타깃 단어를 가지고 주변 문맥 단어가 무엇일지 예측<br>
일반적으로 Skip-gram이 같은 말뭉치로도 더 많은 학습 데이터를 확보할 수 있어 임베딩 품질이 더 좋음

- 네거티브 샘플링 : 타깃 단어와 문맥 단어 쌍이 주어졌을 때 해당 쌍이 포지티브 샘플인지, 네거티브 샘플인지 이진분류하는 과정에서 학습되는 기법<br>
>포지티브 샘플 : 타깃 단어와 그 주변에 실제로 등장한 문맥 단어 쌍<br>
네거티브 샘플 : 타깃 딘아어와 그 주변에 등장하지 않은 단어 쌍

- 서브샘플링 : 자주 등장하는 단어는 학습에서 제외
- Skip-gram 모델은 모델 파라미터를 한 번 업데이트 할 때, 1개 쌍의 포지티브 샘플과 k개의 네거티브 샘플을 학습함

In [9]:
#!git pull origin master

In [4]:
#cd ..

/notebooks/embedding


In [ ]:
#!bash preprocess.sh dump-tokenized

In [6]:
#!cat ../data/tokenized/wiki_ko_mecab.txt ../data/tokenized/ratings_mecab.txt ../data/tokenized/korquad_mecab.txt > ../data/tokenized/corpus_mecab.txt

In [1]:
corpus_fname = "/notebooks/embedding/data/tokenized/corpus_mecab.txt"
model_fname = "/notebooks/embedding/data/word-embeddings/word2vec/word2vec"

from gensim.models import Word2Vec
corpus = [sent.strip().split(" ") for sent in open(corpus_fname, "r").readlines()]
model = Word2Vec(corpus, size=100, workers=4, sg=1)
#size : Word2Vec 임베딩 차원 수
#workers : CPU 스레드 개수
#sg : skip-gram 여부 -> 0이면 CBOW로 학습
model.save(model_fname)

In [3]:
cd ..

/notebooks/embedding


In [4]:
from models.word_eval import WordEmbeddingEvaluator
model = WordEmbeddingEvaluator("/notebooks/embedding/data/word-embeddings/word2vec/word2vec", method="word2vec", dim=100, tokenizer_name="mecab")
model.most_similar("희망", topn=5)

[('소망', 0.7979258),
 ('꿈', 0.7846539),
 ('행복', 0.76131797),
 ('희망찬', 0.7413486),
 ('땀방울', 0.7262753)]

### 4-3. FastText

- FastText는 페이스북에서 개발해 공개한 단어 임베딩 기법
- 각 단어를 문자 단위 n-gram으로 표현하며, 네거티브 샘플링 기법을 사용
- 모델을 한 번 업데이트할 때 1개의 포지티브 샘플과 k개의 네거티브 샘플을 학습
- 오타나 미등록 단어에 강함

In [7]:
!mkdir -p data/word-embeddings/fasttext
!models/fastText/fasttext skipgram -input data/tokenized/corpus_mecab.txt -output data/word-embeddings/fasttext/fasttext

Read 233M words
Number of words:  358043
Number of labels: 0
Progress: 100.0% words/sec/thread:  106732 lr:  0.000000 avg.loss:  0.471360 ETA:   0h 0m 0s 1.751751 ETA:   0h16m42s avg.loss:  1.490483 ETA:   0h15m34s 1.471551 ETA:   0h15m27s avg.loss:  1.469844 ETA:   0h15m23s 1.421293 ETA:   0h14m53s 1.415598 ETA:   0h14m52s ETA:   0h14m49s ETA:   0h14m36s ETA:   0h14m31s 1.379386 ETA:   0h14m20s 1.378151 ETA:   0h14m18s 1.375239 ETA:   0h14m15s  0h14m11s 1.355077 ETA:   0h14m 9s 1.342321 ETA:   0h14m 5s 4s  0h13m55s 1.293545 ETA:   0h13m51s 1.269830 ETA:   0h13m44s 1.261470 ETA:   0h13m41s avg.loss:  1.253478 ETA:   0h13m38s 1.249585 ETA:   0h13m36s words/sec/thread:  104731 lr:  0.043202 avg.loss:  1.221544 ETA:   0h13m23s avg.loss:  1.216588 ETA:   0h13m21s  0h13m19s 1.211285 ETA:   0h13m17s 1.164694 ETA:   0h12m31s 1.158253 ETA:   0h12m24s avg.loss:  1.110803 ETA:   0h12m 6s lr:  0.039228 avg.loss:  1.103464 ETA:   0h12m 4s 1.096447 ETA:   0h12m 2s ETA:   0h11m57s 1.056588 ETA:   0h

In [10]:
from models.word_eval import WordEmbeddingEvaluator
model = WordEmbeddingEvaluator(vecs_txt_fname="data/word-embeddings/fasttext/fasttext.vec",
                              vecs_bin_fname="data/word-embeddings/fasttext/fasttext.bin",
                               method="fasttext", dim=100, tokenizer_name="mecab")
model.most_similar("희망", topn=5)

[('행복', 0.7580105238756496),
 ('소망', 0.7127900391029476),
 ('희망찬', 0.6947392072017382),
 ('기쁨', 0.6851204947805614),
 ('꿈', 0.6769203079901275)]

- fastText는 문자 단위 n-gram을 쓰기 때문에 한글과 궁합이 잘 맞음

**자소 단위로 분해한 데이터로 FastText 시행**<br>
- 은전한닢으로 형태소 분석을 시행
- 이를 자소단위로 분해한 데이터를 만들기
- 이 데이터를 이용해 FastText 임베딩 시행

In [ ]:
#cd ..

In [11]:
from preprocess import jamo_sentence, get_tokenizer
tokenizer = get_tokenizer("mecab")
tokens = " ".join(tokenizer.morphs("나는 학교에 간다"))
print(jamo_sentence(tokens))

ㄴㅏ- ㄴㅡㄴ ㅎㅏㄱㄱㅛ- ㅇㅔ- ㄱㅏㄴㄷㅏ-


In [14]:
!python preprocess/unsupervised_nlputils.py --preprocess_mode jamo \
        --input_path /notebooks/embedding/data/tokenized/corpus_mecab.txt \
        --output_path /notebooks/embedding/data/tokenized/corpus_mecab_jamo.txt

In [15]:
!mkdir -p data/word-embeddings/fasttext-jamo
!models/fastText/fasttext skipgram -input data/tokenized/corpus_mecab_jamo.txt -output data/word-embeddings/fasttext-jamo/fasttext-jamo

Read 233M words
Number of words:  358043
Number of labels: 0
Progress: 100.0% words/sec/thread:   81757 lr:  0.000000 avg.loss:  0.474656 ETA:   0h 0m 0s lr:  0.049927 avg.loss:  1.500141 ETA:   0h21m39s 1.340330 ETA:   0h20m18s 1.396855 ETA:   0h19m58s avg.loss:  1.392696 ETA:   0h19m42s 1.388946 ETA:   0h19m36s words/sec/thread:   80311 lr:  0.048291 avg.loss:  1.386808 ETA:   0h19m31s words/sec/thread:   80419 lr:  0.048214 avg.loss:  1.386763 ETA:   0h19m27s  80456 lr:  0.048131 avg.loss:  1.386841 ETA:   0h19m25s% words/sec/thread:   80843 lr:  0.047610 avg.loss:  1.375748 ETA:   0h19m 6s words/sec/thread:   80814 lr:  0.047456 avg.loss:  1.371540 ETA:   0h19m 3s% words/sec/thread:   80865 lr:  0.047380 avg.loss:  1.364617 ETA:   0h19m 1s% words/sec/thread:   80917 lr:  0.046966 avg.loss:  1.342125 ETA:   0h18m50s% words/sec/thread:   80904 lr:  0.046795 avg.loss:  1.340162 ETA:   0h18m46s words/sec/thread:   80867 lr:  0.046459 avg.loss:  1.335919 ETA:   0h18m38s  80870 lr:  0.04

In [17]:
pwd

'/notebooks/embedding'

In [18]:
from models.word_eval import WordEmbeddingEvaluator
model = WordEmbeddingEvaluator(vecs_txt_fname="data/word-embeddings/fasttext-jamo/fasttext-jamo.vec",
                              vecs_bin_fname="data/word-embeddings/fasttext-jamo/fasttext-jamo.bin",
                               method="fasttext-jamo", dim=100, tokenizer_name="mecab")
model.most_similar("희망", topn=5)

[('희망찬', 0.8165260621686699),
 ('행복', 0.7887172253399581),
 ('희망특강', 0.7397415223317794),
 ('소망', 0.7135493994970037),
 ('희망자', 0.7108970999198214)]

In [19]:
#미등록 단어에 대한 자소 단위 FastText임베딩 결과
model._is_in_vocabulary("서울특별시")
model.get_word_vector("서울특별시")
model.most_similar("서울특별시", topn=5)

[('서울', 0.9475430095956829),
 ('서울특별시', 0.8207505619858799),
 ('특별시세', 0.8071229614705601),
 ('서초구', 0.7877031110761973),
 ('서울시', 0.7874081173655547)]

### 4-4. 잠재 의미 분석

- 커다란 행렬에 차원 축소 방법의 일종인 특이값 분해를 수행하는 기법
- 계산의 효율성을 키우는 한편 행간에 숨어 있는 잠재 의미를 이끌어내기 위한 방법론
- 특이값 분해를 시행한 뒤 결과로 도출되는 행 벡터들을 단어 임베딩으로 상요할 수 있음
- 단어-문서 행렬, TF-IDF 행렬, 단어-문맥 행렬, 점별 상호 정보량(PMI)에 모두 잠재 의미 분석을 수행할 수 있음
- 자연어처리 분야에서는 PMI 대신 PPMI를 사용할 수 있음
>PMI : 두 확률변수 사이의 상관성을 계량화한 지표<br>
PPMI : PMI가 양수가 아닐 경우 그 값을 신뢰하기 어려워 0으로 치환해 무시<br>
Shifted PMI : PMI에서 logk를 빼준 값으로, Word2Vec과 깊은 연관이 있음

- 잠재 의미 분석에 행렬 분해를 이용할 수 있음
>truncated SVD : 특이값 가운데 가장 큰 d개만 가지고 해당 특이값에 대응하는 특이 벡터들로 원래 행렬을 근사하는 방ㅂ버

- LSA를 이용하면 단어와 문맥 간의 내재적인 의미를 효과적으로 보존할 수 있게 돼 문서 간 유사도 측정 등 모델의 성능 향상에 도움을 줄 수 있다고 함
- 또한, 입력 데이터의 노이즈, 희소성을 줄일 수 있다고 함

In [21]:
!cat data/tokenized/ratings_mecab.txt data/tokenized/korquad_mecab.txt > data/tokenized/for-lsa-mecab.txt

In [30]:
#단어-문맥 행렬에 LSA 적용
from sklearn.decomposition import TruncatedSVD
from soynlp.vectorizer import sent_to_word_contexts_matrix

corpus_fname = "/notebooks/embedding/data/tokenized/for-lsa-mecab.txt"

corpus = [sent.replace("\n", "").strip() for sent in open(corpus_fname, "r").readlines()]
input_matrix, idx2vocab = sent_to_word_contexts_matrix(
    corpus,
    windows=3, #타깃 단어 앞뒤 몇 개의 단어를 문맥으로 고려할지
    min_tf=10,
    dynamic_weight=True, #타깃 단어에서 멀어질수록 카운트 동시 등장 점수를 조금씩 깎는다.
    verbose=True) #구축 과정을 화면에 표시할지 여부

cooc_svd = TruncatedSVD(n_components=100) #100차원으로 축소
cooc_vecs = cooc_svd.fit_transform(input_matrix)

Create (word, contexts) matrix
  - counting word frequency from 276937 sents, mem=6.083 Gb
  - scanning (word, context) pairs from 276937 sents, mem=6.083 Gb
  - (word, context) matrix was constructed. shape = (26618, 26618)                    
  - done


In [31]:
#PPMI 행렬에 LSA 적용
from soynlp.word import pmi
ppmi_matrix, _, _ = pmi(input_matrix, min_pmi=0)
ppmi_svd = TruncatedSVD(n_components=100)
ppmi_vecs = ppmi_svd.fit_transform(input_matrix)

In [33]:
#LSA 학습
!python models/word_utils.py --method latent_semantic_analysis \
    --input_path /notebooks/embedding/data/tokenized/for-lsa-mecab.txt \
    --output_path /notebooks/embedding/data/word-embeddings/lsa/lsa

Create (word, contexts) matrix
  - counting word frequency from 276937 sents, mem=0.315 Gb
  - scanning (word, context) pairs from 276937 sents, mem=0.959 Gb
  - (word, context) matrix was constructed. shape = (26618, 26618)                    
  - done


In [39]:
#단어-문맥 행렬 + LSA
from models.word_eval import WordEmbeddingEvaluator
model = WordEmbeddingEvaluator("data/word-embeddings/lsa/lsa-cooc.vecs",
                                method="lsa", dim=100, tokenizer_name="mecab")
model.most_similar("희망", topn=5)

[('진실', 0.948269609730858),
 ('의식', 0.944978793251788),
 ('즐거움', 0.9364362372010614),
 ('인내심', 0.9330672039991788),
 ('사냥', 0.933050890647996)]

In [40]:
#PPMI + LSA
from models.word_eval import WordEmbeddingEvaluator
model = WordEmbeddingEvaluator("data/word-embeddings/lsa/lsa-pmi.vecs",
                              method="lsa", dim=100, tokenizer_name="mecab")
model.most_similar("희망", topn=5)

[('진실', 0.9483008998589746),
 ('의식', 0.9450403597791779),
 ('즐거움', 0.936476402199458),
 ('사냥', 0.9333204748437434),
 ('인내심', 0.9327951308057698)]